In [1]:
!pip install twilio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00


In [2]:
!pip install flask-ngrok
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=bc221f1cd379534b16ba22a021cd3f79c2b6d13f1b8d71846c2485b20b08b81c
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
import os


from twilio.rest import Client

#Twilio account information (save as environment variables)

account_sid = 'YOUR_TWILIO_ACC_SID'
auth_token = 'YOUR_TWILIO_ACC_AUTHTOKEN'
client = Client(account_sid, auth_token)


In [4]:
!pip install -U flask-cors

In [6]:
def call_llama_api(message):
    try:

        payload = {
            'query': message,
        }
        response = requests.post("NGROK_MODEL_ENDPOINT/get", json=payload)
        response.raise_for_status()
        result = response.json()
        if result.get('Response'):
            return result["Response"]
    except Exception as e:
        print(f"Error calling llama-2 API: {e}")

    return 'Error generating response'

In [7]:
!ngrok authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask,jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
import requests
import logging
app = Flask(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)


@app.route('/')
def home():
    return 'All is well...'


@app.route('/twilio/receiveMessage', methods=['POST'])
def receiveMessage():
    try:
        message = request.form['Body']
        sender_id = request.form['From']

        print(sender_id)
        try:
          response = "Test"
          response = call_llama_api(message)
          print(response)
          sent = client.messages.create(
              from_='whatsapp:+14155238886',
              body= response,
              to=sender_id
            )
          print(sent.sid)
        except:
          return jsonify({"Error":"Model error"})
    except:
        pass
    return 'OK', 200



app.run()